In [ ]:
#imports (same as tuto ML)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_20newsgroups
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns
%matplotlib inline

# Question 1: Propensity score matching

We preform a naive data analysis using plots.

In [ ]:
#import the data set
lalonde_df = pd.read_csv('lalonde.csv')
#give a first look
lalonde_df.head()

### 1. A naive analysis

**Looking at numbers**

We use a simple summary to describe the data.

In [ ]:
lalonde_df.groupby('treat')['re78'].describe()

- We see that there are many less people in the treated group compared to the untreated group.
- The standard deviation for the treated group is much higher, there is more variety in the outcome
- The mean of the untreated group is higher
- but the max salary in the treated group is 3x higher!
 - given that the 75% for the untreated group is much higher, this could indicate that the max for the treated group is an outlier

we can visualize these numbers nicely with a boxplot.

In [ ]:
#define the two categories
treat = lalonde_df.treat ==1 
untreat = lalonde_df.treat == 0

treated_salary = lalonde_df[treat]['re78']
untreated_salary = lalonde_df[untreat]['re78']

In [ ]:
plt.boxplot([treated_salary, untreated_salary], labels=['treated', 'untreated'])
plt.show()

**Looking at plots**

we naively (like us at first) try to use a histogram we will see that taking into account that there are much less people in the treated group, both groups fare similarly.

We note that we have a log-like distribution.

In [ ]:
#draw the plots on top of each other with same bin size
bins = np.linspace(0, 60500, 30)
plt.hist(untreated_salary, bins, alpha=0.5, label='untreated')
plt.hist(treated_salary, bins, alpha=0.5, label='treated')
plt.legend(loc='upper right')

**Conclusion**

A naive researcher might conlcude that the treatment isn't effective, as the mean salary for the untreated population is higher, but for some individuals is is very effective, hence the outliers.

### 2. A closer look at the data

We look more closely at the data, building a histogram for every column

In [ ]:
#distinct id is useless
del lalonde_df['id']

In [ ]:
for col in lalonde_df:
    print('Histogram and 5 number summary for column : ', col)
    print('\n',lalonde_df.groupby('treat')[col].describe())
    plt.figure()
    bins = np.linspace(np.min(lalonde_df[col]),np.max(lalonde_df[col]), 30)
    plt.hist(lalonde_df[untreat][col], bins,alpha=0.5)
    plt.hist(lalonde_df[treat][col],bins, alpha=0.5)
    plt.show()

we note different distribution in ...

In [ ]:
for col in ['age', 'educ', 're74', 're75', 're78']:
    plt.figure()
    plt.title(col)
    plt.boxplot([lalonde_df[untreat][col], lalonde_df[treat][col]], labels=['untreated', 'treated'])

Looking at the last 3 diagrams, we see that the treated group is slowly catching up the difference in salary.

We also decide to look at the correlation between the variables. 

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = lalonde_df.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

### 3. A propsensity score model

### 4. Balancing the dataset via matching

### 5. Balancing the groups further


### 6. A less naive analysis

# Question 2: Applied ML

## 1.

Load the 20newsgroup dataset. It is, again, a classic dataset that can directly be loaded using sklearn ([link](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html)).  
[TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), short for term frequency inverse document frequency, is of great help when if comes to compute textual features. Indeed, it gives more importance to terms that are more specific to the considered articles (TF) but reduces the importance of terms that are very frequent in the entire corpus (IDF). Compute TF-IDF features for every article using [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Then, split your dataset into a training, a testing and a validation set (10% for validation and 10% for testing). Each observation should be paired with its corresponding label (the article category).


In [ ]:
# Test and training set are given already, need to choose how to get validation set. Remove as proposed by tutorial
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
# We have the data as a list, transform it to a pandas DF. Not sure if needed though.
train_DF = pd.DataFrame(newsgroups_train.data)
test_DF = pd.DataFrame(newsgroups_test.data)
train_DF.head()

## 2.

Train a random forest on your training set. Try to fine-tune the parameters of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, display a confusion matrix of your classification pipeline. Lastly, once you assessed your model, inspect the `feature_importances_` attribute of your random forest and discuss the obtained results.
